In [262]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

**Achtung**, weil wir einmal neu einsetzen mussten, müssen wird das Ergebnis noch auf Duplikate prüfen

Ansonsten Vorgehen: 
- mit os fileliste rausziehen
- loop bauen mit list comprehension
- df bauen

# Problem:
- Wir haben gemerkt, dass as Zeugs manchmal verrutscht ist. Wir haben versucht, das in der Funktion mit Try und Except in den Griff zu bekommen (indem wir auf andere Zeile zugreifen). Aber das funktioniert nicht. 
- Entweder (vielversprechender): Wir machen die Definitionen ganz neu, inedem wir direkt mit Regex ins soup.find_all("td")[0] einsteigen, wo alle Infos drin sind.
- oder wir versuchen fixe Muster zu finden...

In [317]:
allefiles = os.listdir("simap1_2019")

In [334]:
file = open("simap1_2019/zuschlag768.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

In [335]:
soup

<html xmlns="http://www.w3.org/1999/xhtml"><head>
<title>simap.ch - Einzelmeldungen</title>
<meta content="kps" name="author"/>
<meta content="SIMAP" name="copyright"/>
<meta content="de" http-equiv="Content-Language"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="0" http-equiv="Expires"/>
<meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/>
<meta content="NO-CACHE" http-equiv="PRAGMA"/>
<meta content="INDEX,FOLLOW" name="robots"/>
<meta content="SIMAP" name="keywords"/>
<meta content="Suchergebnis Detailansicht" name="description"/>
<meta content="IE=7" http-equiv="X-UA-Compatible"/>
<script src="/shabforms/COMMON/application/func/fontsize.js" type="text/javascript"></script>
<script src="/shabforms/COMMON/application/func/fokusfields.js" type="text/javascript"></script>
<script src="/shabforms/COMMON/application/func/mm_formhelp.js" type="text/javascript"></script>
<script src="/shabforms/COMMON/application/func/scripts.js" type="text/javascript"></script>
<script s

In [336]:
# Hier ist der Name des Auftraggebers drin:
soup.find_all("td")[0]

<td><h1>Zuschlag</h1><dl><dd><span class="label">Publikationsdatum Simap:</span> 09.03.2018<br/></dd></dl><h2>1. Auftraggeber</h2><h3>1.1 Offizieller Name und Adresse des Auftraggebers</h3><dl><dd><span class="label">Bedarfsstelle/Vergabestelle:</span> Stadt Zürich, Amt für Hochbauten<br/><span class="label">Beschaffungsstelle/Organisator:</span> Stadt Zürich<br/>Amt für Hochbauten, 
		  zu Hdn. von 
		  AHB, Postfach, 
		8021 
		Zürich, 
		Schweiz, 						
			E-Mail: 				
				<a href="mailto:AHB-Beschaffungswesen@zuerich.ch">AHB-Beschaffungswesen@zuerich.ch</a><br/></dd></dl><h3>1.2 Art des Auftraggebers</h3><dl><dd>Gemeinde/Stadt</dd></dl><h3>1.3
			Verfahrensart
		</h3><dl><dd>
					Offenes Verfahren
				<br/></dd></dl><h3>1.4
				Auftragsart
			</h3><dl><dd>
							Bauauftrag
						<br/></dd></dl><h3>1.5
				Gemäss GATT/WTO-Abkommen, resp. Staatsvertrag
			</h3><dl><dd>
						Ja
					<br/></dd></dl><h2>2. Beschaffungsobjekt</h2><h3>2.1 Projekttitel der Beschaffung</h3><dl><dd>BAV 80

In [340]:
#noch genauer:
soup.find_all("td")[0].find_all("dd")[2]

<dd>Gemeinde/Stadt</dd>

In [341]:
#hier holen wir die elenden Zeilenumbrüche und Nonbreakables raus, die Regex stören:

vergabestelle = str(soup.find_all("td")[0].find_all("dd")[2])
vergabestelle = vergabestelle.replace("\n", " ")
vergabestelle = vergabestelle.replace("\xa0", " ") 
vergabestelle

'<dd>Gemeinde/Stadt</dd>'

In [342]:
#Hier haben wir den Namen:
regex1 = r"(?<=<\/span>\s).*(?=<br\/><span)"
vergabestelle = re.findall(regex1, vergabestelle)
vergabestelle

[]

In [343]:
#hier ist das Verfahren
verfahren = (soup.find_all("td")[0].find_all("dd")[4].text)

#Wieder Sonderzeichen rausholen:
verfahren = verfahren.replace("\n", "")
verfahren = verfahren.replace("\t", "")

verfahren

'Bauauftrag'

In [344]:
#hier ist das Projekt
projekt = soup.find_all("td")[0].find_all("dd")[7].text
projekt

'\n\t\t\t\t\t\t\t\tBeschaffungs-Nr\n\t\t\t\t\t\t\t244'

In [326]:
#Projekttyp:
projekttyp = soup.find_all("td")[0].find_all("dd")[8].text

projekttyp = projekttyp.replace("\xa0", " ") 

regex2 = r"(?<=-\s).*"
projekttyp = re.findall(regex2, projekttyp)[0]

projekttyp

'Bauarbeiten für Gebäude im GesundheitswesenBaukostenplannummer (BKP): 2730 - Innentüren aus Holz'

In [329]:
#Oder das hier:
projekttyp = soup.find_all("td")[0].find_all("dd")[10].text

projekttyp = projekttyp.replace("\xa0", " ") 
regex2 = r"(?<=-\s).*(?=\s\(BKP)"
projekttyp = re.findall(regex2, projekttyp)[0]

projekttyp

IndexError: list index out of range

In [330]:
zuschlag2393

NameError: name 'zuschlag2393' is not defined

In [331]:
#Anbieter (Name und Adresse):
anbieter = soup.find_all("td")[0].find_all("dd")[9].text

anbieter = anbieter.replace("\n", "")
anbieter = anbieter.replace("\t", "")
anbieter = anbieter.replace("\xa0", " ")

regex3 = r"(?<=Name:\s).*(?=,)"
anbieter = re.findall(regex3, anbieter)[0]

anbieter = anbieter.split(",")

anbieter_name = anbieter[0].strip()
anbieter_adr1 = anbieter[1].strip()
anbieter_adr2 = anbieter[1].strip()

anbieter_name

'TT Türenfabrik Turbenthal AG'

In [332]:
#ACHTUNG: Geht auch nicht immer. Bei File 2 Ja, bei File 45 nein > Exception:
#Anbieter (Name und Adresse):
anbieter = soup.find_all("td")[0].find_all("dd")[11].text

anbieter = anbieter.replace("\n", "")
anbieter = anbieter.replace("\t", "")
anbieter = anbieter.replace("\xa0", " ")

regex3 = r"(?<=Name:\s).*(?=,)"
anbieter = re.findall(regex3, anbieter)[0]

anbieter = anbieter.split(",")

anbieter_name = anbieter[0].strip()
anbieter_adr1 = anbieter[1].strip()
anbieter_adr2 = anbieter[1].strip()

anbieter_name

IndexError: list index out of range

In [333]:
#Preis:
preis = soup.find_all("td")[0].find_all("dd")[9].text

preis = preis.replace("\n", "")
preis = preis.replace("\t", "")
preis = preis.replace("\xa0", " ")

regex4 = r"\d*'*\d*'*\d*'*\d*\.\d\d"
preis = re.findall(regex4, preis)

preis = float(preis[0].replace("'",""))
preis

140901.0

In [67]:
# List Comprehension

dfs= []
dfs = [pd.read_csv("watson_data/"+str(file)) for file in files]
df = pd.concat(dfs, ignore_index = True)

NameError: name 'files' is not defined